# Модель оценки заголовка

Введенные или сгенерированные заголовки нужно как-то оценивать. Так как главной целью обычно является увеличение количества просмотров, то в качестве критерия нужно использовать число просмотров у ранее опубликованных статей. То есть перед нами стоит задача **регрессии**: на входе заголовок, на выходе число (балл от 0.0 до 10.0 с точностью 0.1).

Для работы с текстами мы используем библиотеку [transformers](https://huggingface.co/transformers/), обладающую высокой эффективностью для задач распознавания особенностей текста (NLU) и его генерации (NLG). Библиотека предоставляет удобный интерфейс для работы с предобученными NLP-моделями на основе архитектуры transformer. Факти это pytorch-модели для NLP-задач, которые легко переводить в tensorflow-модели и обратно.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# standard libraries
import pickle
import io

# data processing libraries
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

# make numpy printouts easier to read
np.set_printoptions(precision=3, suppress=True)

# data processing progress bar
from tqdm.auto import tqdm
tqdm.pandas()

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# deep learning
import torch
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
model_name = 'DeepPavlov/rubert-base-cased'

# пути к датасетам
DATASETS_PATH = "/home/leo/DATASETS"
# TOKENIZED_TITLES_PATH = f"{DATASETS_PATH}/tokenized_titles.pickle"

# общий для приложений словарь с источником данных и их характеристиками
with open('../sources.pickle', 'rb') as f:
    sources = pickle.load(f)

%matplotlib inline

# 1. Подготовка данных
## 1.1. Соединение датафреймов

В нашем распоряжении имеется множество данных, полученных в результате парсинга. Объединим их в один большой датасет для построения модели оценки заголовков. Для построения будем использовать те датасеты (сайты), которые содержат информацию о количестве просмотров статей.

In [2]:
# соединяем датасеты в один общий датасет с именем df
dfs = dict()

for source in sources:
    dfs[source] = pd.read_csv(f"{DATASETS_PATH}/{source}.csv",
                              index_col=0,
                              parse_dates=['post_time', 'parse_time'])
    dfs[source]['source'] = source
    
df = pd.concat(dfs[key] for key in dfs)

# преобразуем количество просмотров
df.views_num = df.views_num.apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))

# удаляем закрытые и недоступные статьи
df = df.drop(df[df.views_num == 0.0].index)

# удаляем дубликаты
df = df.drop_duplicates()
df = df.loc[~df.index.duplicated(keep='last')]
df.head()

,title,post_time,short_text,views_num,parse_time,filename,source,likes_num,favs_num,comments_num
https://tproger.ru/articles/kak-bystro-razvernut-hranilishhe-i-analitiku-dannyh-dlja-biznesa/,Как быстро развернуть хранилище и аналитику да...,2021-03-01 12:32:23+03:00,Сегодня хочу рассказать историю проекта по зап...,7825,2021-03-14,59a7ab25-11f7-502a-b63a-bbdcb121f488,tproger,NaN,NaN,NaN
https://tproger.ru/articles/7-prakticheskih-zadanij-s-sobesedovanija-na-poziciju-junior-java-developer/,7 практических заданий с собеседования на пози...,2021-03-01 09:05:11+03:00,Для начинающего разработчика очень важно не то...,6741,2021-03-14,80f10716-5243-55ca-a67d-4dfe77cd27a5,tproger,NaN,NaN,NaN
https://tproger.ru/quiz/test-chto-mozhet-jeta-nejroset/,"Тест: что реально, а что создала нейросеть?",2021-02-26 19:39:50+03:00,Сегодня нейронные сети используются в сельском...,4032,2021-03-14,aaffd2c5-592f-5d7b-b972-95073d0da49a,tproger,NaN,NaN,NaN
https://tproger.ru/articles/kak-najti-dejstvitelno-horoshij-kurs-po-razrabotke-8-shagov-na-puti-k-pravilnomu-vyboru/,Как найти действительно хороший курс по разраб...,2021-02-26 17:29:00+03:00,Сразу хочется пошутить и предложить разработат...,1121,2021-03-14,16f80dbb-8e7c-5a5b-9025-b2fdef30bfd0,tproger,NaN,NaN,NaN
https://tproger.ru/articles/blackbox-skanery-v-processe-ocenki-bezopasnosti-prilozhenija/,Blackbox-сканеры в процессе оценки безопасност...,2021-02-26 15:16:46+03:00,Профиль задач quality engineer (QE) достаточно...,187,2021-03-14,9d58bde6-7aaf-57a4-b381-25171b9a368f,tproger,NaN,NaN,NaN


## 1.2. Коррекция случаев заниженного числа просмотров
Число просмотров на сайтах иногда существуенно отстает от предполагаемого или не всегда корректно рассчитано. Особенно это заметно на когда количество просмотров меньше числа лайков и добавлений в избранные статьи. Чтобы скорректировать такие значения, построим простую  регрессионную SGD-модель на данных, внушающих доверие и экстраполируем результат на «подозрительные» данные о числе просмотров.

In [3]:
df['post_time'] = pd.to_datetime(df['post_time'], utc=True)
df['mln_secs_to_now'] = (pd.Timestamp.now(tz='UTC') - df['post_time']).apply(lambda x: x.total_seconds())*1e-6
df_tmp = df[['likes_num', 'favs_num', 'comments_num', 'views_num']].dropna()
df_tmp['suspicious'] = [False]*df_tmp.shape[0]
for col in ('likes', 'favs', 'comments'):
    df_tmp['suspicious'] += df_tmp[f'{col}_num'] > 0.1*df_tmp['views_num']

df_tmp_susp = df_tmp[df_tmp['suspicious'] == True]
df_tmp = df_tmp[df_tmp['suspicious'] == False]
df_tmp = df_tmp.drop(columns=['suspicious'])
df_tmp_susp = df_tmp_susp.drop(columns=['suspicious'])

y = df_tmp['views_num']
X = df_tmp.drop(columns=['views_num'])
reg = make_pipeline(StandardScaler(),
                    RandomForestRegressor(n_jobs=20))
reg.fit(X, y)
df_tmp_susp['views_num'] = reg.predict(df_tmp_susp.drop(columns=['views_num']))
df_tmp_susp['views_num'] = df_tmp_susp['views_num'].apply(round)
df_tmp = pd.concat([df_tmp, df_tmp_susp])
df.update(df_tmp)

## 1.3. Преобразование целевой переменной: от числа просмотров к рейтингу
Оставим только данные, которые используются для построения модели: текст заголовка (`X`) и число просмотров (`y`):

In [12]:
Xy = df[['title', 'views_num']]

Отсортируем статьи по количеству просмотров и зададим для каждой позиции рейтинг `score`, равномерно распределенный между 0 для непросматриваемой статьи и 10 для самой просматриваемой. Таким образом, оценка в 9.0 означает, что подобные заголовки имели не меньшее число просмотров, чем 90% иследованного набора данных.

In [14]:
Xy.sort_values(by='views_num', inplace=True)

In [20]:
Xy['score'] = np.linspace(0, 1, Xy.shape[0])
X, y = Xy.title, Xy.score
del Xy

In [21]:
Xy

NameError: name 'Xy' is not defined

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)

In [19]:
X_train

,likes_num,favs_num,comments_num
https://habr.com/ru/post/30662,2.0,147.0,93.0
https://habr.com/ru/post/336572,13.0,69.0,55.0
https://habr.com/ru/post/370295,19.0,161.0,89.0
https://habr.com/ru/post/363933,5.0,17.0,20.0
https://habr.com/ru/post/311664,0.0,8.0,3.0
...,...,...,...
https://habr.com/ru/post/76471,4.0,2.0,8.0
https://habr.com/ru/post/204454,6.0,30.0,7.0
https://habr.com/ru/post/84206,53.0,57.0,90.0
https://habr.com/ru/post/407341,16.0,25.0,26.0


# 2. Построение модели на базе transformers

Начнем с базовой модели: сопоставим каждому заголовку векторное представление и обучим модель для оценки заголовков. 



In [ ]:
max_length = Xy.title.apply(len).max()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# в библиотеке transformers таск sentiment-analysis
# соответствует TextClassificationPipeline
# classifier = pipeline(task="sentiment-analysis",
#                       model=model,
#                       tokenizer=tokenizer)

In [ ]:
max_tokens_in_title = Xy.doc.apply(len).max()
cols_num = 96

def make_stack(doc):
    # представляем заголовок как набор векторов
    words_stack = np.vstack(word.vector for word in doc)
    
    # дополняем плоскость нулями
    zeros_rows_num = max_tokens_in_title-words_stack.shape[0]
    zeros_stack = np.zeros((zeros_rows_num, cols_num))
    plate_stack = np.vstack([words_stack, zeros_stack])
    return plate_stack

In [ ]:
X_train = torch.from_numpy(X_train).float().to(device)

In [ ]:
y_train = torch.from_numpy(y_train).float().to(device)

In [ ]:
y_train

# 4. Проверка дополнительных гипотез

In [ ]:
# генерация дополнительных признаков
# Xy.loc[:, ['title']] = Xy.title.apply(str)

# Xy.loc[:, ['doc']] = Xy.title.progress_apply(nlp)

# длина заголовка в символах
# Xy.loc[:, ['len']] = Xy.title.apply(len)

# количество токенов
# Xy.loc[:, ['tokens_num']] = Xy.tokens.apply(lambda x: len(x))

# XXX Archive & Drafts XXX

In [ ]:
# Токенизация большого числа заголовков — затратная по времени операция.
# Поэтому предварительно токенизированные заголовки хранятся в виде
# сжатого датафрайма
#tokenized_titles = pd.read_pickle(TOKENIZED_TITLES_PATH, compression='gzip')

# for i in [3, 8, 9]:
#     spacy.displacy.render(tokenized_titles.iloc[i], style='ent', jupyter=True)

# tokenized_titles.to_pickle(path=TOKENIZED_TITLES_PATH, compression='gzip')